In [ ]:
%matplotlib widget

import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import ipywidgets as widgets
import json
import numpy as np
import h5py
import multiprocessing
import tempfile

from IPython.display import Video

from ophys_etl.types import ExtractROI
from ophys_etl.modules.segmentation.qc_utils.roi_utils import convert_roi_keys
from ophys_etl.modules.segmentation.qc_utils.video_generator import VideoGenerator
from ophys_etl.modules.segmentation.qc_utils.video_display_generator import VideoDisplayGenerator
from ophys_etl.modules.segmentation.processing_log import SegmentationProcessingLog

from evaldb.reader import EvalDBReader
import support_inspection_nb as support

### **Sqlite DB interface for creating notebook inspection manifest**

We have populated a sqlite database with the metadata associated with each of the 160 SSF experiments, as well as the paths to artifacts generated in the course of segmenting those experiments. The `EvalDBReader` provides an API for accessing that database.

In [ ]:
sqlite_path = Path("/allen/aibs/informatics/segmentation_eval_dbs/ssf_mouse_id_409828.db")
dbreader = EvalDBReader(sqlite_path)

In [ ]:
all_metadata = dbreader.get_all_metadata()
all_metadata

`all_metadata` is a pandas dataframe that can be queried using the pandas API. For instance, to list all of the experiments with a depth between 250 and 350 microns, you can

In [ ]:
all_metadata.query('depth>250 and depth<350')

### **inspection manifest specification**

The above sqlite interface is designed to feed this inspection notebook with an inspection manifest.

The metadata and artifact paths for a given experiment are stored in a dict returned by `dbreader.get_inspection_manifest`. See the example below:

In [ ]:
ophys_experiment_id = 785569447
inspection_manifest = dbreader.get_inspection_manifest(ophys_experiment_id)
pd.DataFrame.from_records([inspection_manifest["metadata"]])

In [ ]:
inspection_manifest



But, if one wants to manually specify inspection data sources, the format is:

```
inspection_manifest = {
    "metadata": dictionary (not required) 
    "videos": list of strings that are paths to videos
    "backgrounds": list of strings that are paths to png or pkl background images/graphs
    "processing_logs": list of strings that are paths to hdf5 processing_logs
```
an empty one:
```
inspection_manifest = {
    "videos": [],
    "backgrounds": [],
    "processing_logs": []}
```

The "processing logs" are the HDF5 files where all of the outputs from the prototype segmentation pipeline are stored. They can be accessed through the API provided by the `SegmentationProcessingLog` class. In this cell, we will get a dict which serves as a lookup table mapping between roi_id and ROIs produced by the `filter` stage of the pipeline.

**Note:** to get the ROIs produced by a different stage in the pipeline, change the first arg of `get_roi_lookup_from_group`. Currently, the only valid group names are "detect" and "filter".

In [ ]:
processing_log_path = f"/allen/aibs/informatics/aamster/ticket_325/detect/{ophys_experiment_id}.h5"
print(processing_log_path)

In [ ]:
processing_log = SegmentationProcessingLog(processing_log_path, read_only=True)
roi_lookup = processing_log.get_roi_lookup_from_group('filter', valid_only=True)

### **ROI Viewer**

The cells below provide a widgetized interface for viewing different segmentations of the same experiment superimposed on different background images.

The columns of the widget allow the user to choose:
- backgrounds: the different projection images used to summarize the videos
- processing logs: these are HDF5 files that contain the results of different segmentations. `*_legacy_from_lims.h5` will contain the legacy segmentation (where available); `{ophys_experiment_id}.h5` will contain the results of the new prototype segmentation.
- dataset: The prototype segmenter runs in multiple stages, each of which produces ROIs. `detect` will specify the ROIs found in the initial detection phase. `filter` will specify the ROIs after a rough quality filter has been applied (**Note:** to select only ROIs that pass the filter, you must check `valid only` for the `filter` dataset).
- Whether or not to display ROI labels and whether or not to limit the display to only ROIs marked as "valid" (all ROIs are marked "valid" by the `detect` phase).

The controls to the left of the widget allow the user to zoom and scroll all of the plots in unison.

In [ ]:
x = widgets.interactive(support.roi_viewer, inspection_manifest=widgets.fixed(inspection_manifest), nrows=[1, 2, 3], ncols=[1, 2, 3]);
display(x)

In [ ]:
# will print out the labels in the zoomed in region
def what_labels_in_zoomed():
    for ax in x.result.axes:
        print("\n" + ax.title.get_text())
        xl = ax.get_xlim()
        yl = ax.get_ylim()[::-1]
        for text in ax.texts:
            tx, ty = text.get_position()
            if (xl[0] <= tx <= xl[1]) & (yl[0] <= ty <= yl[1]):
                print(text.get_text())
what_labels_in_zoomed()

### **Trace Viewer**

The cell below provides a widgetized interface for viewing the traces of ROIs extracted from the available video files. ROI IDs can be exposed by clicking the `include label` selectors in the ROI viewer widget above.

Select the video files from which to extract traces and the ROI whose trace you which to extract (-1 means "no ROI from this dataset").

**Note:** Plotting traces can take several minutes, since the traces have to be read in from the video files.

In [ ]:
rois_dict, trace_widgets, roi_drops, movie_widget_list, trace_grouping = support.get_trace_selection_widgets(inspection_manifest)
display(trace_widgets)
b = widgets.interact_manual(support.trace_plot_callback,
                            rois_dict=widgets.fixed(rois_dict),
                            roi_drops=widgets.fixed(roi_drops),
                            movie_widget_list=widgets.fixed(movie_widget_list),
                            trace_grouping=widgets.fixed(trace_grouping),
                            description="plot traces")
b.widget.children[0].description = "plot traces"

### **Video viewing**

The cells below provide classes and functions to view thumbnail videos in this notebook.

The interface relies on two classes. `VideoGenerator` creates instances of the class `ThumbnailVideo` which point to videos stored on disk. `VideoDisplayGenerator` reads `ThumbnailVideo` instances and converts them into kwargs that can be passed to IPython's `Video` API.


In [ ]:
display_generator = VideoDisplayGenerator()

The `inspection_manifest` lists the available video files under `"videos"`.

In [ ]:
movie_list = inspection_manifest["videos"]

In [ ]:
movie_list

We will now instantiate two `VideoGenerator` instances: one for the noisy video; one for the denoised video.

This will take a few minutes as the `VideoGenerator` scans the entire video file to find a contrast-enhancing normalization for the video data.

In [ ]:
noisy_movie_path = '/allen/programs/braintv/production/neuralcoding/prod55/specimen_734689833/ophys_session_785378984/ophys_experiment_785569447/processed/785569447_suite2p_motion_output.h5'
denoised_movie_path = '/allen/programs/braintv/workgroups/nc-ophys/danielk/deepinterpolation/experiments/ophys_experiment_785569447/denoised.h5'

In [ ]:
%%time
noisy_video_generator = VideoGenerator(noisy_movie_path)
denoised_video_generator = VideoGenerator(denoised_movie_path)

As an illustration, we will select the ROI with ID 8

In [ ]:
roi_id = 8

The `VideoGenerator` instances include a method to generate thumbnail videos centered on an ROI, either with, or without the ROI overplotted. We will now create a thumbnail video from the denoised movie with the ROI boundary overplotted in red. To do this, pass the ROI as the first argument of `get_thumbnail_video_from_roi`.

This will also take a few minutes, as the video is generated and written to disk.

In [ ]:
%%time
example_thumbnail = denoised_video_generator.get_thumbnail_video_from_roi(roi_lookup[roi_id], roi_color=(255, 0, 0), quality=9)

To display the video, we use the `display_generator` and IPython's `Video` API

In [ ]:
Video(**display_generator.display_video(example_thumbnail))

To see where the video has been written to disk, use its `video_path` property (this will be a seemingly random filename generated using Python's `tempfile` API).

**Note:** once a `ThumbnailVideo` instance passes out of scope, its video on disk is deleted. If you want to save a thumbnail video for later use, you need to copy it before the associated `ThumbnailVideo` instance is deleted.

In [ ]:
example_thumbnail.video_path

`get_thumbnail_video_from_roi` allows you to focus on a specific subset of timesteps using the `timestep` kwarg. From the trace plot above, we know that timesteps `[8500, 10500]` are interesting for this ROI. We will generate thumbnail videos focusing exclusively on that window in time below.

In [ ]:
%%time
timesteps = np.arange(8500, 10500)

noisy_video_with_roi = noisy_video_generator.get_thumbnail_video_from_roi(roi_lookup[roi_id], roi_color=(255, 0, 0), quality=9,
                                                                          timesteps=timesteps)

denoised_video_with_roi = denoised_video_generator.get_thumbnail_video_from_roi(roi_lookup[roi_id], roi_color=(255, 0, 0), quality=9,
                                                                                timesteps=timesteps)

# this video will show the same field of view without the ROI superimposed over it
noisy_video_without_roi = noisy_video_generator.get_thumbnail_video_from_roi(roi_lookup[roi_id], roi_color=None, quality=9,
                                                                             timesteps=timesteps)

In [ ]:
Video(**display_generator.display_video(noisy_video_with_roi))

In [ ]:
Video(**display_generator.display_video(noisy_video_without_roi))

In [ ]:
Video(**display_generator.display_video(denoised_video_with_roi))

It is also possible to generate a thumbnail video by specifying an origin and a field of view shape. This method also accepts the `timesteps` kwarg.

**Note:** this API requires you to specify `origin` and `frame_shape`. All coordinates and dimensions are listed using the image processing convention `(row, column)`, which is effectively `(y, x)`. We apologize for the confusion.

In [ ]:
%%time
timesteps = np.arange(3700, 5500)
by_hand_thumbnail = denoised_video_generator.get_thumbnail_video(origin=(100, 60),
                                                                 frame_shape=(64, 64),
                                                                 quality=9,
                                                                 timesteps=timesteps)

In [ ]:
%%time
Video(**display_generator.display_video(by_hand_thumbnail))

To see the same region with all overlapping ROIs overplotted, you can pass in a dict or list of ROIs using the `rois` kwarg. Use `valid_only` to make sure you only plot ROIs that are flagged as "valid".

**Note:** using `valid_only` is somewhat redundant in our example, since we used `valid_only=True` when constructing our roi lookup dict.

In [ ]:
%%time
timesteps = np.arange(3700, 5500)
by_hand_thumbnail_with_rois = denoised_video_generator.get_thumbnail_video(
                                                                 origin=(100, 60),
                                                                 frame_shape=(64, 64),
                                                                 quality=9,
                                                                 timesteps=timesteps,
                                                                 rois=roi_lookup,
                                                                 valid_only=True)

In [ ]:
%%time
Video(**display_generator.display_video(by_hand_thumbnail_with_rois))